In [2]:
# ===============================================
# 0️⃣ Google Drive 마운트
# ===============================================
from google.colab import drive
drive.mount('/content/drive')

# 경로 설정
BASE_PATH = "/content/drive/MyDrive/dwililiyafood101-model-classification"
MODEL_PTH = f"{BASE_PATH}/best_model.pth"

# ===============================================
# 1️⃣ 환경 설정
# ===============================================
!pip install torch torchvision torchaudio
!pip install onnx onnx-tf tensorflow tensorflowjs

# ===============================================
# 2️⃣ PyTorch EfficientNet-B0 로드 및 ONNX로 변환
# ===============================================
import torch
import torchvision.models as models

# EfficientNet-B0 불러오기 (pretrained 구조 사용)
model = models.efficientnet_b0(pretrained=False)
model.classifier[1] = torch.nn.Linear(model.classifier[1].in_features, 101)  # Food-101

# 가중치 로드
state_dict = torch.load(MODEL_PTH, map_location="cpu")
model.load_state_dict(state_dict)
model.eval()

# 더미 입력 (3×224×224)
dummy_input = torch.randn(1, 3, 224, 224)

# PyTorch → ONNX 변환
torch.onnx.export(
    model,
    dummy_input,
    f"{BASE_PATH}/model.onnx",
    input_names=["input"],
    output_names=["output"],
    opset_version=11,
)
print("✅ [1단계 완료] PyTorch → ONNX 변환 완료: model.onnx 생성")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
INFO: pip is looking at multiple versions of onnx-tf to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 186.6/186.6 kB 17.8 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipython-input-2770823717.py:36: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.ht

✅ [1단계 완료] PyTorch → ONNX 변환 완료: model.onnx 생성
